In [1]:
#!pip install scrapy_proxies
!pip install scrapy-fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13489 sha256=ce4d127febb5e2ab87b4e7f13c1d9cec99ced67420e5d0585f5ba4ff154d3046
  Stored in directory: c:\users\diego\appdata\local\pip\cache\wheels\ae\e7\76\7dd44644d065268ab0e1b4fa2e802fa4bb0157717b7d6c6d92
Successfully built fake-useragent


In [1]:
import scrapy
import csv
import time
import scrapy_proxies
from scrapy.crawler import CrawlerProcess
import pandas as pd
#import scrapy-fake-useragent
class FirstWebsiteSpider(scrapy.Spider):
    
    rotate_user_agent = True   
    name = 'first_website'
    #https://www.amazon.com/s?k=data+science&i=stripbooks-intl-ship&page=3&crid=1YWU6OJK4L3BA&qid=1662895059&sprefix=data+science%2Cstripbooks-intl-ship%2C247&ref=sr_pg_3
    allowed_domains = ["www.amazon.com"] #, "https://www.amazon.com"
    handle_httpstatus_list = [503]
    start_urls = ["http://www.amazon.com/s?k=statistics&crid=34NU2WDUOI6UY&qid=1663448396&sprefix=statisti%2Caps%2C263&ref=sr_pg_1"]
    list_items=[]
    j=1
    #download_delay = 3 #The number you write here will be how many seconds scrapy waits before sending another request.
    #x=pd.DataFrame([])
    n_page_to_scrape=7
    items_in_page=60
    RETRY_TIMES=20
    #DOWNLOADER_MIDDLEWARES = {
    #'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
    #'scrapy.downloadermiddlewares.retry.RetryMiddleware': None,
    #'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
    #'scrapy_fake_useragent.middleware.RetryUserAgentMiddleware': 401,
    #}
    FAKEUSERAGENT_PROVIDERS = [
    'scrapy_fake_useragent.providers.FakeUserAgentProvider',  # this is the first provider we'll try
    'scrapy_fake_useragent.providers.FakerProvider',  # if FakeUserAgentProvider fails, we'll use faker to generate a user-agent string for us
    'scrapy_fake_useragent.providers.FixedUserAgentProvider',  # fall back to USER_AGENT value
]
    def parse(self, response):
        print (response.request.headers['User-Agent'])
    def parse(self, response):
        #self.j=j
        print(self.j)
        #time.sleep(3)
        next_page_url=f"https://www.amazon.com/s?k=statistics&page={self.j}&crid=34NU2WDUOI6UY&qid=1663448390&sprefix=statisti%2Caps%2C263&ref=sr_pg_{self.j}"
        box=response.xpath("//div[@class='a-section a-spacing-small puis-padding-left-small puis-padding-right-small']")             #("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-3']")
        #print(box)
                                 
        #links=[]
        #titles=[]
        item={}
        
        for ind,i in enumerate(box):
            #print('g')
            #country_name = country.xpath(".//text()").get()
            #title=i.xpath("//span[@class='a-size-base a-color-base a-text-normal']/text()").getall()
            title=i.xpath(".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']/span/text()").get()
            link = i.xpath(".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']/@href").get()
            author=i.xpath(".//div[@class='a-row a-size-base a-color-secondary']/a[@class='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style']/text() | (.//div[@class='a-row a-size-base a-color-secondary']/a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style']/text()) | (.//div[@class='a-row a-size-base a-color-secondary']/span[@class='a-size-base']/text())").getall()
            
            #links.append(link)
            #'a-link-normal s-underline-text s-underline-link-text s-link-style'
            #titles.append(title)
            print(title)
            #//div[@class='a-section a-spacing-small puis-padding-left-small puis-padding-right-small']/*/*/a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']/@href  #print(title)
            yield response.follow(url=link, callback=self.parse_book, meta={'ind':ind,'title':title,'author':author,'link':link})
    
  
    # Getting data inside the "link" website
    def parse_book(self, response):
        #time.sleep(4)
        # Getting country names and each row element inside the population table
        title = response.request.meta['title']
        ind = response.request.meta['ind']
        link = response.request.meta['link']
        author = response.request.meta['author']
        #if author==[' ']:
        price=response.xpath("//div[@id='buyNew_cbb']/div[@class='a-row']/div[@class='a-column a-span12 a-text-left']/span/text() | ((//a[@class='a-link-normal']/span[@class='a-color-price']/text())[1]) | (//a[@class='a-button-text']/span[@class='a-color-base']/span[@class='a-size-base a-color-price a-color-price']/text())").get()
        #//div[@id='buyNew_cbb']/div[@class='a-row']/div[@class='a-column a-span12 a-text-left']/span/text() | ((//a[@class='a-link-normal']/span[@class='a-color-price']/text())[1]) 
        #pages=response.xpath("//span[@class='a-list-item']/*[contains(text(),'pages')/text()").get()
        pages=response.xpath("//span[@class='a-list-item']/span[contains( text(),'pages')]/text()").get()
        language=response.xpath("//span[@class='a-text-bold'][contains(text(),'Language')]/following-sibling::span[1]/text()").get()
        publisher=response.xpath("//span[@class='a-text-bold'][contains(text(),'Publisher')]/following-sibling::span[1]/text()").get()
        dimensions=response.xpath("//span[@class='a-text-bold'][contains(text(),'Dimensions')]/following-sibling::span[1]/text()").get()
        weight=response.xpath("//span[@class='a-text-bold'][contains(text(),'Weight')]/following-sibling::span[1]/text()").get()
        avg_reviews=response.xpath("(//a[@class='a-popover-trigger a-declarative']/i[@class='a-icon a-icon-star a-star-5']/span[@class='a-icon-alt']/text())[1] | ((//a[@class='a-popover-trigger a-declarative']/i/span[@class='a-icon-alt'])[1]/text())").get()
        time.sleep(0.1)
        star5=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[1]/text()").getall()
        star4=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[2]/text()").getall()
        star3=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[3]/text()").getall()
        star2=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[4]/text()").getall()
        star1=response.xpath("(//span[@class='a-size-base']/a[contains(text(),'%')])[5]/text()").getall()
        ISBN_13=response.xpath("//div[@id='isbn_feature_div']/div/div/span[contains(text(),'13')]/following-sibling::span[1]/text()").get()
        n_reviews=response.xpath("(//span[@class='a-declarative']/a[@class='a-link-normal']/span[@id='acrCustomerReviewText']/text())[1]").get()
        
        #print(title)
        #author= response.xpath("//span[@class='a-declarative']/a[@class='a-link-normal contributorNameID']/text()").get()  # You can also use the whole class value  --> response.xpath('(//table[@class="table table-striped table-bordered table-hover table-condensed table-list"])[1]/tbody/tr')
            # Return data extracted
        keys=['title','author','price','pages','avg_reviews','n_reviews','star5','star4','star3','star2','star1','dimensions','weight','language','publisher','ISBN_13','link']
        values= [title,author,price,pages,avg_reviews,n_reviews,star5,star4,star3,star2,star1,dimensions,weight,language,publisher,ISBN_13,link]    #  {'title': title, 'author':author}
        self.list_items.append(values)
        item=dict(zip(keys, values))
        print(len(self.list_items))
        print(star5,avg_reviews,dimensions)
        #print(title,author)
        
      
        #if self.j >1:
          #  next_page_url=f"https://www.amazon.com/s?k=data+science&s=relevanceblender&page={self.j}&crid=2YKZLS0CJFDGI&qid=1663103507&sprefix=data+science%2Caps%2C828&ref=sr_pg_{self.j}"
        #else:
            #next_page_url=self.start_urls[0]
        #yield scrapy.Request(url=next_page_url, callback=self.parse_pagination, meta={'title':title,'author':author})
        
        #   def parse_pagination(self,response):
        #title = response.request.meta['title']  
        #author=response.request.meta['author'] 
        #print('va beeeeeneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee')
        #if ind==
        #print(ind)
        
        if ind==59:
            time.sleep(3)
            self.j+=1
            next_page_url=f"https://www.amazon.com/s?k=statistics&page={self.j}&crid=34NU2WDUOI6UY&qid=1663448390&sprefix=statisti%2Caps%2C263&ref=sr_pg_{self.j}"
            if next_page_url and self.j<self.n_page_to_scrape:
                print('start_new_page')
            
                yield scrapy.Request(url=next_page_url, callback=self.parse)#,self.j 
            #time.sleep(10)
         #,'authors','dates'      
        if len(self.list_items)==(self.items_in_page): #self.j==self.n_page_to_scrape:
            time.sleep(5)
            print(self.list_items)
            x=pd.DataFrame(self.list_items,columns=keys)
            #print(x)
            yield x.to_csv("outputfile2_statistics.csv",sep=",")
            #with open("outputfile.csv", "w", newline="") as f:
                #writer = csv.DictWriter(f, ['titles','authors','dates'])
                #writer.writeheader()           
                #writer.writerows(self.list_items)
                #print(self.list_items) #
        if len(self.list_items)==225:#(self.n_page_to_scrape-2)*(self.items_in_page): #self.j==self.n_page_to_scrape:
            #time.sleep(70)
            #print(self.list_items)
            y=pd.DataFrame(self.list_items,columns=keys)
            #print(x)
            yield y.to_csv("outputfile_final_statistics.csv",sep=",")
            #with open("outputfile.csv", "w", newline="") as f:
                #writer = csv.DictWriter(f, ['titles','authors','dates'])
                #writer.writeheader()           
                #writer.writerows(self.list_items)
                #print(self.list_items) #
                
#//div[@class='a-row']/a[@class='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style']/text() | ( //div[@class='a-#row']/span[@class='a-size-base']/text()
process = CrawlerProcess({
     #'USER_AGENT':'USER_AGENT':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    #'USER_AGENT':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Safari/605.1.15"#
    #'USER_AGENT':'Mozilla/5.0 (compatible; MSIE 7.0; Windows NT 5.1)'
  'USER_AGENT':  "Mozilla/4.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36",
  #'USER_AGENT':  "Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36",
  #'USER_AGENT':  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 Safari/603.3.8",
  #'USER_AGENT':  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15"
  #'USER_AGENT':  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15"
  #'USER_AGENT':  "Mozilla/5.0 (Windows NT 10.0; Windows; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"
  #'USER_AGENT':  "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36"




})

process.crawl(FirstWebsiteSpider)
process.start()


2022-09-18 00:16:23 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-09-18 00:16:24 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 3.4.7, Platform Windows-10-10.0.22000-SP0
2022-09-18 00:16:24 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Chrome/103.0.5060.53 Safari/537.36'}
2022-09-18 00:16:24 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-09-18 00:16:24 [scrapy.extensions.telnet] INFO: Telnet Password: 3928ebfc8a5f5cf2
2022-09-18 00:16:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2022

1
Introduction to Statistics: An Intuitive Guide for Analyzing Data and Unlocking Discoveries
Princeton Review AP Statistics Prep, 2022: 4 Practice Tests + Complete Content Review + Strategies & Techniques (2022) (College Test Preparation)
Introduction to Probability, Statistics, and Random Processes
Statistics for Absolute Beginners (Second Edition) (Data Science, Python & Statistics for Beginners)
Practical Statistics for Data Scientists: 50+ Essential Concepts Using R and Python
Naked Statistics: Stripping the Dread from the Data
The Art of Statistics: How to Learn from Data
Statistics Laminate Reference Chart: Parameters, Variables, Intervals, Proportions (Quickstudy: Academic )
Statistics For Dummies (For Dummies (Lifestyle))
Statistics for People Who (Think They) Hate Statistics
Essential Math for Data Science: Take Control of Your Data with Fundamental Linear Algebra, Probability, and Statistics
Introductory Statistics
How to Lie with Statistics
Advanced Statistics for Health Re

2022-09-18 00:16:27 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (302) to <GET https://www.amazon.com/Introduction-Statistics-Intuitive-Analyzing-Discoveries/dp/1735431109/ref=sr_1_1_sspa?crid=34NU2WDUOI6UY&keywords=statistics&qid=1663452985&sprefix=statisti%2Caps%2C263&sr=8-1-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEzS05CV1ZCT0ZDWk1UJmVuY3J5cHRlZElkPUEwMzAxNTc0MVExSklKNEpKTkUzTyZlbmNyeXB0ZWRBZElkPUEwNDYyMDI5MjYwMk9MMlg2STg0RyZ3aWRnZXROYW1lPXNwX2F0ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU=> from <GET https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A04620292602OL2X6I84G&url=%2FIntroduction-Statistics-Intuitive-Analyzing-Discoveries%2Fdp%2F1735431109%2Fref%3Dsr_1_1_sspa%3Fcrid%3D34NU2WDUOI6UY%26keywords%3Dstatistics%26qid%3D1663452985%26sprefix%3Dstatisti%252Caps%252C263%26sr%3D8-1-spons%26psc%3D1&qualifier=1663452985&id=4337139339647806&widgetName=sp_atf>
2022-09-18 00:16:27 [scrapy.downloadermiddlewares.re

1
['\n            \n              79%\n            \n          '] 4.5 out of 5 stars 7 x 0.75 x 9 inches


2022-09-18 00:16:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Statistics-Laminate-Reference-Chart-Proportions/dp/1572229446/ref=sr_1_8?crid=34NU2WDUOI6UY&keywords=statistics&qid=1663452985&sprefix=statisti%2Caps%2C263&sr=8-8> (referer: https://www.amazon.com/s?k=statistics&crid=34NU2WDUOI6UY&qid=1663448396&sprefix=statisti%2Caps%2C263&ref=sr_pg_1)
2022-09-18 00:16:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Practical-Statistics-Data-Scientists-Essential/dp/149207294X/ref=sr_1_5?crid=34NU2WDUOI6UY&keywords=statistics&qid=1663452985&sprefix=statisti%2Caps%2C263&sr=8-5> (referer: https://www.amazon.com/s?k=statistics&crid=34NU2WDUOI6UY&qid=1663448396&sprefix=statisti%2Caps%2C263&ref=sr_pg_1)
2022-09-18 00:16:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Statistics-People-Think-They-Hate/dp/1544381859/ref=sr_1_10?crid=34NU2WDUOI6UY&keywords=statistics&qid=1663452985&sprefix=statisti%2Caps%2C263&sr=8-10> (re

...

235
['\n            \n              78%\n            \n          '] 4.6 out of 5 stars 6 x 0.86 x 9 inches
236
['\n            \n              73%\n            \n          '] 4.4 out of 5 stars None
237
['\n            \n              62%\n            \n          '] 4.2 out of 5 stars 5.5 x 0.9 x 8.4 inches


2022-09-18 00:18:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Clinical-Biostatistics-Epidemiology-Ridiculously-Simple/dp/1935660020/ref=sr_1_18_sspa?crid=34NU2WDUOI6UY&keywords=statistics&qid=1663452985&sprefix=statisti%2Caps%2C263&sr=8-18-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEzS05CV1ZCT0ZDWk1UJmVuY3J5cHRlZElkPUEwMzAxNTc0MVExSklKNEpKTkUzTyZlbmNyeXB0ZWRBZElkPUEwODQ3NzMwMjBXN0JORkJETjhWRSZ3aWRnZXROYW1lPXNwX210ZiZhY3Rpb249Y2xpY2tSZWRpcmVjdCZkb05vdExvZ0NsaWNrPXRydWU=> (referer: https://www.amazon.com/s?k=statistics&crid=34NU2WDUOI6UY&qid=1663448396&sprefix=statisti%2Caps%2C263&ref=sr_pg_1)
2022-09-18 00:18:06 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.amazon.com/Learn-Statistics-Today-Equations-Included/dp/1793436789/ref=sr_1_19_sspa?crid=34NU2WDUOI6UY&keywords=statistics&qid=1663452985&sprefix=statisti%2Caps%2C263&sr=8-19-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEzS05CV1ZCT0ZDWk1UJmVuY3J5cHRlZElkPUEwMzAxNTc0MVExSklKNEpKTkUzTyZlbmNyeXB0ZW

238
['\n            \n              70%\n            \n          '] 4.5 out of 5 stars 5.75 x 0.25 x 8.75 inches


2022-09-18 00:18:06 [scrapy.core.engine] INFO: Closing spider (finished)
2022-09-18 00:18:06 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 288914,
 'downloader/request_count': 291,
 'downloader/request_method_count/GET': 291,
 'downloader/response_bytes': 59326308,
 'downloader/response_count': 291,
 'downloader/response_status_count/200': 243,
 'downloader/response_status_count/301': 1,
 'downloader/response_status_count/302': 47,
 'elapsed_time_seconds': 102.12154,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2022, 9, 17, 22, 18, 6, 799062),
 'httpcompression/response_bytes': 305038542,
 'httpcompression/response_count': 243,
 'log_count/DEBUG': 296,
 'log_count/ERROR': 1,
 'log_count/INFO': 11,
 'request_depth_max': 7,
 'response_received_count': 243,
 'scheduler/dequeued': 291,
 'scheduler/dequeued/memory': 291,
 'scheduler/enqueued': 291,
 'scheduler/enqueued/memory': 291,
 'spider_exceptions/PermissionError': 1,
 'start_time'

239
['\n            \n              60%\n            \n          '] 4.2 out of 5 stars 6 x 0.14 x 9 inches


In [2]:
authors,titles=FirstWebsiteSpider.parse(scrapy.Spider,response)

NameError: name 'response' is not defined

In [ ]:
@href='/s?k=data+science&amp;i=stripbooks-intl-ship&amp;page=3&amp;crid=20YNDMUXOLCJ6&amp;qid=1663030048&amp;sprefix=data+science%2Cstripbooks-intl-ship%2C314&amp;ref=sr_pg_3' 

In [2]:
j=1
p=f'/s?k=data+science&amp;i=stripbooks-intl-ship&amp;page={j}&amp;crid=20YNDMUXOLCJ6&amp;qid=1663030048&amp;sprefix=data+science%2Cstripbooks-intl-ship%2C314&amp;ref=sr_pg_{j}'

In [3]:
p

'/s?k=data+science&amp;i=stripbooks-intl-ship&amp;page=1&amp;crid=20YNDMUXOLCJ6&amp;qid=1663030048&amp;sprefix=data+science%2Cstripbooks-intl-ship%2C314&amp;ref=sr_pg_1'

In [ ]:
<a href=>3</a>

In [3]:
u=0
a=f'rit{u}'
a

'rit0'

In [4]:
u=1
a

'rit0'

[3]

In [ ]:
#print(item)    
        self.j+=1
        print(self.j)
        next_page_url=f"https://www.amazon.com/s?k=data+science&s=relevanceblender&page={self.j}&crid=2YKZLS0CJFDGI&qid=1663103507&sprefix=data+science%2Caps%2C828&ref=sr_pg_{self.j}"
        if next_page_url and self.j<self.n_page_to_scrape:
            print('non va bene')
            yield response.follow(url=next_page_url, callback=self.parse)#,self.j  
         #,'authors','dates'
        if self.j==self.n_page_to_scrape:
            #print(self.list_items)
            x=pd.DataFrame(self.list_items,columns=['title','author'])
            #print(x)
            yield x.to_csv("outputfile2.csv",sep=",")
        #yield response.follow(url=next_page_url, callback=self.parse_pagination, meta=item )